In [1]:
import re
import pandas as pd
from textblob import TextBlob
from textblob.exceptions import NotTranslated

FILE_NAME = 'magazine_tweets.csv'

In [2]:
company_tweets = pd.read_csv(FILE_NAME, sep=',', names=['id', 'text'])
company_tweets.head()
company_tweets.drop_duplicates(subset='text', inplace=True)
company_tweets.sort_values('id', inplace=True)

In [3]:
company_tweets_clean = pd.DataFrame(columns=('id', 'date', 'text', 'text_en', 'sentiment_polarity', 'sentiment_subjectivity'))

for i in company_tweets.index:
    if company_tweets.at[i, 'id'] > 368:
        full_text = company_tweets.at[i, 'text']
        date, tweet = full_text.split("Text=")
        date_splitted = date.split(' ')
        
        if date_splitted[1] == "Sep":
            new_date_format = "2019-09-{:s} {:s}".format(date_splitted[2], date_splitted[3])
        elif date_splitted[1] == "Oct":
            new_date_format = "2019-10-{:s} {:s}".format(date_splitted[2], date_splitted[3])
        else:
            new_date_format = "2019-11-{:s} {:s}".format(date_splitted[2], date_splitted[3])
        
        tweet = ''.join(tweet.split('\\n'))
        clean_tweet = ' '.join(re.sub("(@[A-Za-z0-9-ZÀ-ÿ]+)|([^0-9A-Za-z-ZÀ-ÿ \t])|(\w+://\S+)", " ", tweet).split())
        
        phrase = TextBlob(clean_tweet)
        try:
            translate = TextBlob(str(phrase.translate(from_lang='pt', to='en_US')))
            single_row = {
                         'id': company_tweets.at[i, 'id'],
                         'date': new_date_format,
                         'text': str(phrase),
                         'text_en': str(translate),
                         'sentiment_polarity': translate.sentiment.polarity,
                         'sentiment_subjectivity': translate.sentiment.subjectivity
                          }
            company_tweets_clean = company_tweets_clean.append(single_row, ignore_index=True)
        except NotTranslated:
                print("Invalid tweet {:s}!".format(str(phrase)))

Invalid tweet YouTube Lu!
Invalid tweet Magalu!


HTTPError: HTTP Error 429: Too Many Requests

In [6]:
final = pd.read_csv(FILE_NAME.split('.')[0] + "_clean.csv", sep=',')
final.drop(final.columns[[0]], axis=1, inplace=True)
final.head()

,id,date,text,text_en,sentiment_polarity,sentiment_subjectivity
0,1,2019-09-10 16:52:08,me diz como faz quando o frete é quase metade ...,tell me how you do it when freight is almost h...,-0.166667,0.166667
1,2,2019-09-10 16:54:51,Já tentou cancelar alguma coisa por telefone N...,Already tried to cancel something over the pho...,0.218434,0.571465
2,3,2019-09-10 17:05:36,Com o lançamento do Iphone novo podia rolar aq...,With the launch of the new Iphone could roll t...,0.211039,0.495130
3,4,2019-09-10 17:08:00,RT Começou gente Tim Cook está no palco do Ste...,RT Started People Tim Cook is on stage at the ...,0.000000,0.000000
4,5,2019-09-10 17:11:34,Lu vai me atualizando Obrigado,Lu is updating me Thank you,0.000000,0.000000


In [8]:
merged_data = pd.concat([final, company_tweets_clean])
print(merged_data.to_string())

      id                 date                                               text                                            text_en  sentiment_polarity  sentiment_subjectivity
0      1  2019-09-10 16:52:08  me diz como faz quando o frete é quase metade ...  tell me how you do it when freight is almost h...           -0.166667                0.166667
1      2  2019-09-10 16:54:51  Já tentou cancelar alguma coisa por telefone N...  Already tried to cancel something over the pho...            0.218434                0.571465
2      3  2019-09-10 17:05:36  Com o lançamento do Iphone novo podia rolar aq...  With the launch of the new Iphone could roll t...            0.211039                0.495130
3      4  2019-09-10 17:08:00  RT Começou gente Tim Cook está no palco do Ste...  RT Started People Tim Cook is on stage at the ...            0.000000                0.000000
4      5  2019-09-10 17:11:34                     Lu vai me atualizando Obrigado                        Lu is updating m

In [9]:
merged_data.to_csv(FILE_NAME.split('.')[0] + "_clean_2.csv", sep=',', encoding='utf-8')
